<a href="https://colab.research.google.com/github/MLDreamer/AIMathematicallyexplained/blob/main/Claude_animation_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""


INCLUDES HERO GIF: The attention matrix explosion that hooks readers instantly

Run this in Google Colab for best results.
Install: !pip install manim matplotlib numpy pillow
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle, FancyBboxPatch, Circle, FancyArrowPatch, Polygon
from matplotlib.collections import LineCollection
import matplotlib.patches as mpatches
from IPython.display import HTML
import io
from PIL import Image

# Set style
plt.style.use('dark_background')
BLUE = '#3b82f6'
CYAN = '#06b6d4'
RED = '#ef4444'
GREEN = '#10b981'
PURPLE = '#a855f7'
YELLOW = '#fbbf24'
ORANGE = '#f97316'

# ============================================
# HERO GIF: THE ATTENTION MATRIX SHOWDOWN
# ============================================

def create_hero_gif():
    """
    THE MONEY SHOT: Side-by-side comparison of GPT vs Claude
    Shows GPT's matrix exploding while Claude stays compact
    This is what goes at the TOP of every article
    """
    fig = plt.figure(figsize=(16, 9), facecolor='#0f172a')
    gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

    ax_gpt = fig.add_subplot(gs[0, 0])
    ax_claude = fig.add_subplot(gs[0, 1])
    ax_graph = fig.add_subplot(gs[1, :])

    def animate(frame):
        ax_gpt.clear()
        ax_claude.clear()
        ax_graph.clear()

        # Progress through sequence lengths
        n = int(10 + (frame / 120) * 190)  # 10 to 200 tokens
        progress = frame / 120

        # ===== LEFT: GPT's EXPLODING MATRIX =====
        ax_gpt.set_xlim(0, 210)
        ax_gpt.set_ylim(0, 210)
        ax_gpt.set_aspect('equal')
        ax_gpt.set_facecolor('#1a0a0a')  # Dark red tint

        # Draw exploding matrix
        matrix = np.random.rand(min(n, 200), min(n, 200)) * 0.4
        im_gpt = ax_gpt.imshow(matrix, cmap='Reds', extent=[0, n, 0, n],
                               alpha=0.7, vmin=0, vmax=1)

        # Danger border - gets more intense
        border_color = RED if n > 100 else ORANGE
        border_width = 3 + (n / 20)
        ax_gpt.add_patch(Rectangle((0, 0), n, n, fill=False,
                                   edgecolor=border_color,
                                   linewidth=border_width, alpha=0.9))

        # "MEMORY OVERFLOW" warning
        if n > 150:
            ax_gpt.text(n/2, n/2, '⚠️ MEMORY\nOVERFLOW',
                       ha='center', va='center',
                       fontsize=20, color=RED, weight='bold',
                       bbox=dict(boxstyle='round', facecolor='black',
                                alpha=0.8, edgecolor=RED, linewidth=3))

        # Labels
        ax_gpt.set_title(f'GPT-5: O(n²) Attention\n{n}×{n} = {n*n:,} operations',
                        fontsize=14, color=RED, weight='bold', pad=10)
        ax_gpt.set_xlabel('Keys', fontsize=11, color='white')
        ax_gpt.set_ylabel('Queries', fontsize=11, color='white')
        ax_gpt.tick_params(colors='white', labelsize=8)

        # ===== RIGHT: CLAUDE'S COMPRESSED BEAUTY =====
        ax_claude.set_xlim(0, 210)
        ax_claude.set_ylim(0, 210)
        ax_claude.set_aspect('equal')
        ax_claude.set_facecolor('#0a1a0a')  # Dark green tint

        # Compressed latent representation (stays small!)
        latent_size = 32  # Fixed small size!
        latent_matrix = np.random.rand(latent_size, latent_size) * 0.5
        im_claude = ax_claude.imshow(latent_matrix, cmap='Greens',
                                     extent=[0, latent_size, 0, latent_size],
                                     alpha=0.8, vmin=0, vmax=1)

        # Glowing border
        for i in range(3):
            ax_claude.add_patch(Rectangle((0-i*2, 0-i*2),
                                         latent_size+i*4, latent_size+i*4,
                                         fill=False, edgecolor=GREEN,
                                         linewidth=2, alpha=0.4-i*0.1))

        # Core box
        ax_claude.add_patch(Rectangle((0, 0), latent_size, latent_size,
                                     fill=False, edgecolor=GREEN,
                                     linewidth=4, alpha=1))

        # Compression annotation
        ax_claude.text(latent_size + 20, latent_size/2,
                      f'Compressed\nLatent Core\n\n{latent_size}×{latent_size}\n= {latent_size*latent_size:,} ops',
                      fontsize=12, color=GREEN, weight='bold',
                      va='center',
                      bbox=dict(boxstyle='round', facecolor='black',
                               alpha=0.7, edgecolor=GREEN, linewidth=2))

        # Show "original" size lightly in background
        ax_claude.add_patch(Rectangle((0, 0), n, n, fill=False,
                                     edgecolor='white', linewidth=1,
                                     alpha=0.2, linestyle='--'))
        ax_claude.text(n-10, n-10, f'{n}×{n}\noriginal',
                      fontsize=9, color='white', alpha=0.4, ha='right')

        # Labels
        ax_claude.set_title(f'Claude 4: O(n) Latent Attention\nConstant memory footprint',
                           fontsize=14, color=GREEN, weight='bold', pad=10)
        ax_claude.set_xlabel('Compressed Dimensions', fontsize=11, color='white')
        ax_claude.set_ylabel('Latent Space', fontsize=11, color='white')
        ax_claude.tick_params(colors='white', labelsize=8)

        # ===== BOTTOM: PERFORMANCE COMPARISON GRAPH =====
        ax_graph.set_facecolor('#0a0a1a')

        # Data points
        x_vals = np.arange(10, n+1, 5)

        # GPT: Quadratic growth
        y_gpt = x_vals ** 2
        ax_graph.plot(x_vals, y_gpt, color=RED, linewidth=4,
                     label='GPT-5: O(n²)', alpha=0.9)
        ax_graph.scatter([n], [n**2], color=RED, s=300, zorder=5,
                        edgecolor='white', linewidth=2)

        # Claude: Linear growth (with latent dimension)
        d_latent = 512
        y_claude = x_vals * d_latent
        ax_graph.plot(x_vals, y_claude, color=GREEN, linewidth=4,
                     label='Claude: O(n·512)', alpha=0.9)
        ax_graph.scatter([n], [n*d_latent], color=GREEN, s=300, zorder=5,
                        edgecolor='white', linewidth=2)

        # Crossover point annotation
        if n > 80:
            speedup = (n**2) / (n*d_latent)
            ax_graph.annotate(f'{speedup:.0f}x slower!\n\nGPT: {n**2:,} ops\nClaude: {n*d_latent:,} ops',
                            xy=(n, n**2), xytext=(n-40, n**2*0.6),
                            fontsize=12, color=RED, weight='bold',
                            bbox=dict(boxstyle='round', facecolor='black',
                                     alpha=0.9, edgecolor=RED, linewidth=2),
                            arrowprops=dict(arrowstyle='->', color=RED,
                                          lw=3, connectionstyle='arc3,rad=0.3'))

        # Styling
        ax_graph.set_xlabel('Sequence Length (lines of code)',
                           fontsize=13, color='white', weight='bold')
        ax_graph.set_ylabel('Computational Operations',
                           fontsize=13, color='white', weight='bold')
        ax_graph.set_title('The Mathematical Proof: Why Claude Dominates Large Codebases',
                          fontsize=16, color=CYAN, weight='bold', pad=15)
        ax_graph.legend(fontsize=12, loc='upper left', framealpha=0.9)
        ax_graph.grid(True, alpha=0.2, linestyle='--')
        ax_graph.tick_params(colors='white', labelsize=10)

        # Add "The Disaster Zone" shading
        if n > 100:
            ax_graph.axvspan(100, 200, alpha=0.1, color=RED,
                           label='GPT Crash Zone')
            ax_graph.text(150, ax_graph.get_ylim()[1]*0.9,
                         'GPT MEMORY\nOVERFLOW →',
                         fontsize=11, color=RED, weight='bold', ha='center',
                         bbox=dict(boxstyle='round', facecolor='black',
                                  alpha=0.7, edgecolor=RED))

        # Progress indicator
        fig.text(0.5, 0.02,
                f'Analyzing codebase: {n} lines | Frame {frame+1}/120',
                ha='center', fontsize=11, color=CYAN, weight='bold')

        plt.tight_layout()

    anim = animation.FuncAnimation(fig, animate, frames=120, interval=50)
    anim.save('HERO_attention_showdown.gif', writer='pillow', fps=24, dpi=120)
    print("✅ HERO GIF SAVED: HERO_attention_showdown.gif")
    print("   📌 Use this as the FIRST image in LinkedIn and Medium!")
    plt.close()

# ============================================
# ANIMATION 1: QUADRATIC ATTENTION EXPLOSION
# ============================================

def create_quadratic_explosion_gif():
    """
    Shows how attention complexity explodes quadratically
    Visual: Matrix growing from 10x10 to 100x100 with calculation count
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), facecolor='#0f172a')

    def animate(frame):
        ax1.clear()
        ax2.clear()

        # Progress through sequence lengths
        n = int(10 + (frame / 100) * 90)  # 10 to 100 tokens

        # Left: Show attention matrix
        ax1.set_xlim(0, 100)
        ax1.set_ylim(0, 100)
        ax1.set_aspect('equal')
        ax1.set_facecolor('#1a0a0a')

        # Draw matrix grid
        matrix = np.random.rand(n, n) * 0.3
        ax1.imshow(matrix, cmap='Reds', extent=[0, n, 0, n], alpha=0.8)

        # Highlight quadratic growth
        color = RED if n > 70 else ORANGE if n > 40 else BLUE
        ax1.add_patch(Rectangle((0, 0), n, n, fill=False,
                                edgecolor=color, linewidth=3))

        ax1.set_title(f'Attention Matrix: {n}×{n} tokens',
                     fontsize=16, color=color, pad=20, weight='bold')
        ax1.set_xlabel('Keys', fontsize=12, color='white')
        ax1.set_ylabel('Queries', fontsize=12, color='white')
        ax1.tick_params(colors='white')

        # Right: Show computation count
        x_vals = np.arange(10, n+1)
        y_vals = x_vals ** 2  # Quadratic

        ax2.set_facecolor('#0a1a0a')
        ax2.plot(x_vals, y_vals, color=RED, linewidth=3, label='O(n²) - GPT')
        ax2.scatter([n], [n**2], color=RED, s=200, zorder=5)

        # Add linear comparison (Claude's approach)
        y_linear = x_vals * 512  # O(n·d_c) where d_c=512
        ax2.plot(x_vals, y_linear, color=GREEN, linewidth=3,
                linestyle='--', label='O(n·d_c) - Claude', alpha=0.7)

        ax2.set_xlabel('Sequence Length (tokens)', fontsize=12, color='white')
        ax2.set_ylabel('Computations', fontsize=12, color='white')
        ax2.set_title(f'Operations: {n**2:,} vs {n*512:,}',
                     fontsize=16, color=CYAN, pad=20, weight='bold')
        ax2.legend(fontsize=10, loc='upper left')
        ax2.grid(True, alpha=0.2)
        ax2.tick_params(colors='white')

        # Annotate the disaster
        if n > 50:
            speedup = (n**2) / (n*512)
            ax2.annotate(f'GPT: {n**2:,} ops\n{speedup:.0f}x slower!',
                        xy=(n, n**2), xytext=(n-20, n**2*0.7),
                        fontsize=11, color=RED, weight='bold',
                        bbox=dict(boxstyle='round', facecolor=RED, alpha=0.3),
                        arrowprops=dict(arrowstyle='->', color=RED, lw=2))

        fig.tight_layout()

    anim = animation.FuncAnimation(fig, animate, frames=100, interval=50)
    anim.save('quadratic_explosion.gif', writer='pillow', fps=20, dpi=100)
    print("✓ Saved: quadratic_explosion.gif")
    plt.close()

# ============================================
# ANIMATION 2: KV CACHE COMPRESSION
# ============================================

def create_kv_compression_gif():
    """
    Shows compression of massive KV cache into tiny latent vector
    Visual: Large matrix flowing through funnel into small glowing orb
    """
    fig, ax = plt.subplots(figsize=(12, 8), facecolor='#0f172a')

    def animate(frame):
        ax.clear()
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 10)
        ax.axis('off')
        ax.set_facecolor('#0a0a1a')

        progress = frame / 100

        # Original KV Cache (large)
        kv_width = 3
        kv_height = 4
        kv_rect = FancyBboxPatch((1, 5), kv_width, kv_height,
                                 boxstyle="round,pad=0.1",
                                 edgecolor=BLUE, facecolor=BLUE,
                                 alpha=0.3, linewidth=3)
        ax.add_patch(kv_rect)
        ax.text(2.5, 7, 'KV Cache\n4096 dims\n1.6 GB', ha='center', va='center',
               fontsize=12, color='white', weight='bold')

        # Compression funnel with gradient
        funnel_x = [4, 5, 5, 4]
        funnel_y = [7 - progress*2, 7 - progress*2, 5, 5]

        # Multiple funnel layers for depth
        for i in range(3):
            offset = i * 0.1
            ax.fill([x + offset for x in funnel_x], funnel_y,
                   color=CYAN, alpha=0.3-i*0.1)
            ax.plot([x + offset for x in funnel_x], funnel_y,
                   color=CYAN, linewidth=2, alpha=0.7-i*0.2)

        # Compression formula
        if progress > 0.3:
            ax.text(4.5, 8.5, r'$c_{KV} = W_{DKV} \cdot h_t$',
                   fontsize=14, color=CYAN, ha='center',
                   bbox=dict(boxstyle='round', facecolor='black',
                            alpha=0.8, edgecolor=CYAN, linewidth=2))

        # Compressed latent (small, glowing)
        if progress > 0.5:
            glow_alpha = (progress - 0.5) * 2
            latent_size = 0.3 + (progress - 0.5) * 0.4

            # Multiple glow layers
            for i in range(4):
                circle = Circle((6.5, 5), latent_size + i*0.12,
                              color=GREEN, alpha=glow_alpha * (0.6 - i*0.15))
                ax.add_patch(circle)

            # Core
            circle = Circle((6.5, 5), latent_size,
                          color=GREEN, alpha=1)
            ax.add_patch(circle)
            ax.text(6.5, 3.5, 'Latent\n512 dims\n200 MB', ha='center',
                   fontsize=11, color=GREEN, weight='bold')

        # Reconstruction arrows (if fully compressed)
        if progress > 0.8:
            recon_progress = (progress - 0.8) * 5
            arrow = FancyArrowPatch((6.5 + recon_progress*0.5, 5),
                                   (6.5 + recon_progress*1.5, 5),
                                   arrowstyle='->', mutation_scale=30,
                                   color=PURPLE, linewidth=3, alpha=recon_progress)
            ax.add_patch(arrow)

            if recon_progress > 0.5:
                ax.text(8.5, 5.5, 'Reconstruct\non-demand',
                       fontsize=10, color=PURPLE, style='italic', weight='bold')

        # Compression ratio
        if progress > 0.6:
            ax.text(5, 2, f'Compression: 8x\nMemory: 1.6GB → 200MB\nSavings: 88%',
                   fontsize=13, color=GREEN, ha='center', weight='bold',
                   bbox=dict(boxstyle='round', facecolor='black', alpha=0.8,
                            edgecolor=GREEN, linewidth=2))

        # Title
        ax.text(5, 9.5, 'Multi-Head Latent Attention: The Compression Miracle',
               ha='center', fontsize=16, color=CYAN, weight='bold')

        ax.set_aspect('equal')

    anim = animation.FuncAnimation(fig, animate, frames=100, interval=50)
    anim.save('kv_compression.gif', writer='pillow', fps=20, dpi=100)
    print("✓ Saved: kv_compression.gif")
    plt.close()

# ============================================
# ANIMATION 3: LATENT SPACE RECONSTRUCTION
# ============================================

def create_reconstruction_gif():
    """
    Shows how compressed latent vector reconstructs full attention
    Visual: Small orb expands into multiple heads
    """
    fig, ax = plt.subplots(figsize=(12, 8), facecolor='#0f172a')

    def animate(frame):
        ax.clear()
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 10)
        ax.axis('off')
        ax.set_facecolor('#0a0a1a')

        progress = frame / 100

        # Central latent vector
        latent_x, latent_y = 2, 5

        # Pulsing glow
        pulse = 0.1 * np.sin(frame * 0.2)
        for i in range(3):
            circle = Circle((latent_x, latent_y), 0.4 + i*0.1 + pulse,
                          color=GREEN, alpha=0.4-i*0.1)
            ax.add_patch(circle)

        circle = Circle((latent_x, latent_y), 0.4, color=GREEN, alpha=1)
        ax.add_patch(circle)
        ax.text(latent_x, 3.8, r'$c_{KV}$', ha='center',
               fontsize=14, color=GREEN, weight='bold')

        # Multi-head reconstruction
        num_heads = 8
        angles = np.linspace(0, 2*np.pi, num_heads, endpoint=False)
        radius = 3

        for i, angle in enumerate(angles):
            if progress > i / num_heads:
                head_progress = min(1, (progress - i/num_heads) * num_heads)

                # Position
                x = latent_x + radius * np.cos(angle) * head_progress
                y = latent_y + radius * np.sin(angle) * head_progress

                # Arrow from latent to head
                arrow = FancyArrowPatch((latent_x, latent_y), (x, y),
                                       arrowstyle='->', mutation_scale=20,
                                       color=BLUE, linewidth=2, alpha=head_progress)
                ax.add_patch(arrow)

                # Head representation
                head_size = 0.3 * head_progress
                head = Circle((x, y), head_size, color=BLUE, alpha=0.7)
                ax.add_patch(head)

                if head_progress > 0.8:
                    ax.text(x, y, f'H{i+1}', ha='center', va='center',
                           fontsize=9, color='white', weight='bold')

        # Mathematical formulas
        if progress > 0.3:
            ax.text(8, 8.5, r'$K_{rope} = W_{UK} \cdot c_{KV}$',
                   fontsize=13, color=BLUE,
                   bbox=dict(boxstyle='round', facecolor='black', alpha=0.8,
                            edgecolor=BLUE, linewidth=2))
            ax.text(8, 7.5, r'$V = W_{UV} \cdot c_{KV}$',
                   fontsize=13, color=CYAN,
                   bbox=dict(boxstyle='round', facecolor='black', alpha=0.8,
                            edgecolor=CYAN, linewidth=2))

        # Title
        ax.text(5, 9.5, 'On-Demand Reconstruction: 96 Heads from 1 Latent Core',
               ha='center', fontsize=16, color=CYAN, weight='bold')

        if progress > 0.8:
            ax.text(5, 1, '8 heads × shared latent = 73x memory savings',
                   fontsize=12, color=GREEN, ha='center', weight='bold',
                   bbox=dict(boxstyle='round', facecolor='black', alpha=0.8,
                            edgecolor=GREEN, linewidth=2))

    anim = animation.FuncAnimation(fig, animate, frames=100, interval=50)
    anim.save('latent_reconstruction.gif', writer='pillow', fps=20, dpi=100)
    print("✓ Saved: latent_reconstruction.gif")
    plt.close()

# ============================================
# ANIMATION 4: SYSTEM 1 VS SYSTEM 2 REASONING
# ============================================

def create_reasoning_comparison_gif():
    """
    Shows accuracy improvement during Extended Thinking
    Visual: Two paths - GPT (fast, flat) vs Claude (slow, climbing)
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), facecolor='#0f172a')

    def animate(frame):
        ax1.clear()
        ax2.clear()

        time = np.linspace(0, 5, 100)
        progress = frame / 100
        current_time = time[:frame+1]

        ax1.set_facecolor('#0a0a1a')
        ax2.set_facecolor('#0a0a1a')

        # System 1 (GPT): Fast but plateaus
        gpt_accuracy = 60 + 20 * (1 - np.exp(-current_time * 3))
        ax1.plot(current_time, gpt_accuracy, color=RED, linewidth=3, label='GPT-5 (System 1)')
        ax1.scatter([current_time[-1]], [gpt_accuracy[-1]], color=RED, s=200, zorder=5)

        # System 2 (Claude): Slower but keeps improving
        claude_accuracy = 50 + 45 * (1 - np.exp(-current_time * 0.8))
        ax1.plot(current_time, claude_accuracy, color=BLUE, linewidth=3, label='Claude (System 2)')
        ax1.scatter([current_time[-1]], [claude_accuracy[-1]], color=BLUE, s=200, zorder=5)

        ax1.set_xlim(0, 5)
        ax1.set_ylim(0, 100)
        ax1.set_xlabel('Time (seconds)', fontsize=12, color='white')
        ax1.set_ylabel('Code Correctness (%)', fontsize=12, color='white')
        ax1.set_title('System 1 vs System 2 Reasoning', fontsize=16, color=CYAN, pad=20, weight='bold')
        ax1.legend(fontsize=11, loc='lower right')
        ax1.grid(True, alpha=0.3)
        ax1.axhline(y=80, color='gray', linestyle='--', alpha=0.5)
        ax1.text(4.5, 82, 'Production threshold', fontsize=9, color='gray')
        ax1.tick_params(colors='white')

        if progress > 0.5:
            ax1.annotate('Extended Thinking pays off',
                        xy=(2.5, claude_accuracy[50]), xytext=(3.5, 85),
                        fontsize=11, color=BLUE, weight='bold',
                        bbox=dict(boxstyle='round', facecolor=BLUE, alpha=0.3),
                        arrowprops=dict(arrowstyle='->', color=BLUE, lw=2))

        # Bottom: Show reasoning stages
        stages = ['Parse', 'Plan', 'Simulate', 'Critique', 'Refine', 'Output']
        stage_times = [0, 0.8, 1.6, 2.8, 4.0, 5.0]

        for i, (stage, t) in enumerate(zip(stages, stage_times)):
            if current_time[-1] >= t:
                alpha = min(1, (current_time[-1] - t) * 2)
                color = BLUE if i < len(stages)-1 else GREEN

                rect = FancyBboxPatch((i*1.5, 0.2), 1.2, 0.6,
                                     boxstyle="round,pad=0.05",
                                     edgecolor=color, facecolor=color,
                                     alpha=alpha*0.4, linewidth=2)
                ax2.add_patch(rect)
                ax2.text(i*1.5 + 0.6, 0.5, stage, ha='center', va='center',
                        fontsize=11, color='white', weight='bold', alpha=alpha)

                if i > 0 and current_time[-1] >= stage_times[i-1]:
                    arrow = FancyArrowPatch(((i-1)*1.5+1.2, 0.5), (i*1.5, 0.5),
                                           arrowstyle='->', mutation_scale=20,
                                           color=CYAN, linewidth=2, alpha=alpha)
                    ax2.add_patch(arrow)

        ax2.set_xlim(-0.5, 10)
        ax2.set_ylim(0, 1)
        ax2.axis('off')
        ax2.set_title('Claude System 2 Pipeline', fontsize=14, color=CYAN, pad=20, weight='bold')

        fig.tight_layout()

    anim = animation.FuncAnimation(fig, animate, frames=100, interval=50)
    anim.save('system2_reasoning.gif', writer='pillow', fps=20, dpi=100)
    print("✓ Saved: system2_reasoning.gif")
    plt.close()

# ============================================
# ANIMATION 5: REPOSITORY NAVIGATION
# ============================================

def create_codebase_navigation_gif():
    """
    Shows Claude navigating a codebase with latent space memory
    Visual: Network graph where nodes light up as dependencies are traced
    """
    fig, ax = plt.subplots(figsize=(12, 10), facecolor='#0f172a')

    # Define a simple dependency graph
    nodes = {
        'api/checkout': (2, 7),
        'lib/validator': (5, 8),
        'middleware/auth': (5, 5),
        'types/commerce': (8, 6),
        'utils/payment': (8, 9),
        'db/models': (2, 4),
        'config/env': (5, 2)
    }

    edges = [
        ('api/checkout', 'lib/validator'),
        ('api/checkout', 'middleware/auth'),
        ('lib/validator', 'types/commerce'),
        ('middleware/auth', 'types/commerce'),
        ('middleware/auth', 'db/models'),
        ('api/checkout', 'utils/payment'),
        ('utils/payment', 'config/env'),
        ('db/models', 'config/env')
    ]

    def animate(frame):
        ax.clear()
        ax.set_xlim(0, 10)
        ax.set_ylim(0, 10)
        ax.axis('off')
        ax.set_facecolor('#0a0a1a')

        progress = frame / 100
        active_node_idx = int(progress * len(nodes))

        # Draw edges
        for i, (src, dst) in enumerate(edges):
            if i < active_node_idx * 2:
                x1, y1 = nodes[src]
                x2, y2 = nodes[dst]
                alpha = min(1, (active_node_idx * 2 - i) * 0.3)
                ax.plot([x1, x2], [y1, y2], color=CYAN, linewidth=2, alpha=alpha)

        # Draw nodes
        for i, (name, (x, y)) in enumerate(nodes.items()):
            if i <= active_node_idx:
                # Active or visited
                alpha = 1 if i == active_node_idx else 0.5
                color = GREEN if i == active_node_idx else BLUE

                # Glow effect for active node
                if i == active_node_idx:
                    for j in range(3):
                        circle = Circle((x, y), 0.4 + j*0.1, color=color, alpha=0.3-j*0.1)
                        ax.add_patch(circle)

                circle = Circle((x, y), 0.3, color=color, alpha=alpha)
                ax.add_patch(circle)
                ax.text(x, y-0.7, name, ha='center', fontsize=9, color='white', alpha=alpha)
            else:
                # Not yet visited
                circle = Circle((x, y), 0.3, color='gray', alpha=0.3)
                ax.add_patch(circle)
                ax.text(x, y-0.7, name, ha='center', fontsize=9, color='gray', alpha=0.3)

        # Title and annotations
        ax.text(5, 9.5, 'Latent Space Code Navigation', ha='center',
               fontsize=18, color=CYAN, weight='bold')

        if active_node_idx > 0:
            current_file = list(nodes.keys())[active_node_idx]
            ax.text(5, 0.5, f'Analyzing: {current_file}',
                   fontsize=13, color=GREEN, ha='center', weight='bold',
                   bbox=dict(boxstyle='round', facecolor='black', alpha=0.8))

        # Memory indicator
        memory_used = (active_node_idx / len(nodes)) * 51  # MB
        ax.text(9, 9, f'Memory: {memory_used:.1f}MB',
               fontsize=11, color=BLUE, ha='right',
               bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

    anim = animation.FuncAnimation(fig, animate, frames=100, interval=100)
    anim.save('codebase_navigation.gif', writer='pillow', fps=10, dpi=100)
    print("✓ Saved: codebase_navigation.gif")
    plt.close()

# ============================================
# MAIN EXECUTION
# ============================================

if __name__ == "__main__":
    print("🎬 Generating 3Blue1Brown Style Animations...")
    print("=" * 60)

    print("\n🏆 HERO GIF (LinkedIn/Medium Top Image)...")
    create_hero_gif()

    print("\n1️⃣  Creating Quadratic Explosion Animation...")
    create_quadratic_explosion_gif()

    print("\n2️⃣  Creating KV Cache Compression Animation...")
    create_kv_compression_gif()

    print("\n3️⃣  Creating Latent Reconstruction Animation...")
    create_reconstruction_gif()

    print("\n4️⃣  Creating System 2 Reasoning Animation...")
    create_reasoning_comparison_gif()

    print("\n5️⃣  Creating Codebase Navigation Animation...")
    create_codebase_navigation_gif()

    print("\n" + "=" * 60)
    print("✅ ALL ANIMATIONS GENERATED!")
    print("\n📥 Downloadable files:")
    print("   🏆 HERO_attention_showdown.gif (USE THIS FIRST!)")
    print("   • quadratic_explosion.gif")
    print("   • kv_compression.gif")
    print("   • latent_reconstruction.gif")
    print("   • system2_reasoning.gif")
    print("   • codebase_navigation.gif")
    print("\n📍 Visual Cue Placement:")
    print("   [HERO] HERO_attention_showdown.gif → TOP of every article!")
    print("   [CUE 1] quadratic_explosion.gif → After 'Django Moment'")
    print("   [CUE 2] kv_compression.gif → After 'Compression Miracle'")
    print("   [CUE 3] latent_reconstruction.gif → After 'Multi-Head Magic'")
    print("   [CUE 4] system2_reasoning.gif → After 'System 2 Reasoning'")
    print("   [CUE 5] codebase_navigation.gif → After 'Pattern Interrupt'")
    print("\n🎨 Upload to Imgur/Medium and embed as images!")
    print("\n💡 PRO TIP: The HERO gif is designed to be:")
    print("   • Instantly eye-catching (red vs green)")
    print("   • Self-explanatory (shows the math)")
    print("   • Share-worthy (perfect for social media)")
    print("   • Hook for LinkedIn (stops the scroll!)")

🎬 Generating 3Blue1Brown Style Animations...

🏆 HERO GIF (LinkedIn/Medium Top Image)...


/tmp/ipython-input-185717139.py:195: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


✅ HERO GIF SAVED: HERO_attention_showdown.gif
   📌 Use this as the FIRST image in LinkedIn and Medium!

1️⃣  Creating Quadratic Explosion Animation...
✓ Saved: quadratic_explosion.gif

2️⃣  Creating KV Cache Compression Animation...
✓ Saved: kv_compression.gif

3️⃣  Creating Latent Reconstruction Animation...
✓ Saved: latent_reconstruction.gif

4️⃣  Creating System 2 Reasoning Animation...
✓ Saved: system2_reasoning.gif

5️⃣  Creating Codebase Navigation Animation...
✓ Saved: codebase_navigation.gif

✅ ALL ANIMATIONS GENERATED!

📥 Downloadable files:
   🏆 HERO_attention_showdown.gif (USE THIS FIRST!)
   • quadratic_explosion.gif
   • kv_compression.gif
   • latent_reconstruction.gif
   • system2_reasoning.gif
   • codebase_navigation.gif

📍 Visual Cue Placement:
   [HERO] HERO_attention_showdown.gif → TOP of every article!
   [CUE 1] quadratic_explosion.gif → After 'Django Moment'
   [CUE 2] kv_compression.gif → After 'Compression Miracle'
   [CUE 3] latent_reconstruction.gif → After '